# Evaluación Módulo 2 Sprint 2 Mónica Serrano

## Ejercicios con código final en funciones.

In [105]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import mysql.connector

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:

In [106]:
lista = ['Canada','Argentina','United States']

In [107]:
def sacar_informacion(lista_paises):
    """Esta función saca toda la información de una api  de los paises definidos.
        Args:
            lista_paises(list): lista de paises del cual queremos saber toda la información.
        return: aunque esta función no devuelve nada, genera un dataframe con la información actualizada de los paises seleccionados.
        """
    df = pd.DataFrame()
    for pais in lista_paises:
        api_url = f"http://universities.hipolabs.com/search?country={pais}"
        response = requests.get(api_url)
        if response.status_code != 200:
            print(response.reason)
            print(response.status_code)
            break
        df_pais = pd.json_normalize(response.json())
        df = pd.concat([df, df_pais], axis = 0)
    return df

In [108]:
df = sacar_informacion(lista)

In [109]:
df.head()

,web_pages,alpha_two_code,state-province,name,domains,country
0,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",CA,Quebec,Cégep de Saint-Jérôme,[cstj.qc.ca],Canada
1,[https://www.lambtoncollege.ca],CA,Ontario,Lambton College,"[lambtoncollege.ca, mylambton.ca]",Canada
2,[http://www.acadiau.ca/],CA,Nova Scotia,Acadia University,[acadiau.ca],Canada
3,[http://www.algonquincollege.com/],CA,Ontario,Algonquin College,[algonquincollege.com],Canada
4,[http://www.ashtoncollege.com/],CA,British Columbia,Ashton College,[ashtoncollege.com],Canada


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

- Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.

In [110]:
nuevas_columna = {col: col.replace("-", "_") for col in df.columns}
df.rename(columns= nuevas_columna, inplace = True)

In [111]:
df.head()

,web_pages,alpha_two_code,state_province,name,domains,country
0,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",CA,Quebec,Cégep de Saint-Jérôme,[cstj.qc.ca],Canada
1,[https://www.lambtoncollege.ca],CA,Ontario,Lambton College,"[lambtoncollege.ca, mylambton.ca]",Canada
2,[http://www.acadiau.ca/],CA,Nova Scotia,Acadia University,[acadiau.ca],Canada
3,[http://www.algonquincollege.com/],CA,Ontario,Algonquin College,[algonquincollege.com],Canada
4,[http://www.ashtoncollege.com/],CA,British Columbia,Ashton College,[ashtoncollege.com],Canada


- La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

In [112]:
df.drop('domains',axis=1,inplace=True)

In [113]:
df.head()

,web_pages,alpha_two_code,state_province,name,country
0,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",CA,Quebec,Cégep de Saint-Jérôme,Canada
1,[https://www.lambtoncollege.ca],CA,Ontario,Lambton College,Canada
2,[http://www.acadiau.ca/],CA,Nova Scotia,Acadia University,Canada
3,[http://www.algonquincollege.com/],CA,Ontario,Algonquin College,Canada
4,[http://www.ashtoncollege.com/],CA,British Columbia,Ashton College,Canada


3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva.

In [114]:
df.shape

(5008, 5)

In [115]:
df = df.explode('web_pages')

In [116]:
df.head()

,web_pages,alpha_two_code,state_province,name,country
0,https://www.cstj.qc.ca,CA,Quebec,Cégep de Saint-Jérôme,Canada
0,https://ccmt.cstj.qc.ca,CA,Quebec,Cégep de Saint-Jérôme,Canada
0,https://ccml.cstj.qc.ca,CA,Quebec,Cégep de Saint-Jérôme,Canada
1,https://www.lambtoncollege.ca,CA,Ontario,Lambton College,Canada
2,http://www.acadiau.ca/,CA,Nova Scotia,Acadia University,Canada


In [117]:
df.shape

(5034, 5)

4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [118]:
df.duplicated(subset='name').sum()

2543

In [119]:
df.drop_duplicates(subset='name',inplace=True)

In [120]:
df.duplicated(subset='name').sum()

0

5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [121]:
df['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', None,
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán', 'Pennsylvania',
       'Texas', 'Utah', 'NV', 'Iowa', 'VA', 'TX', 'Indiana', 'Colorado',
       'Ohio', 'IN', 'New York', 'CA', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Virginia', 'Washington',
       'Missouri', 'California', 'NY', 'ND', 'MI', 'Florida', 'Michigan',
       'GA', 'New York, NY', 'Maine'], dtype=object)

In [122]:
def cambiar_nulos(dataframe,columna):
    """
    Esta función cambia los nulos de pandas por nulos de numpy.

    Args:
        dataframe (dataframe): el dataframe que contiene los nulos de pandas
        columna (str): la columna la cual queremos modificar.
    return: aunque esta función no devuelve nada, cambia el tipo de nulo de una columna especifica."""
    dataframe[columna].replace({None},np.nan,inplace=True)

In [123]:
cambiar_nulos(df,"state_province")

In [124]:
df['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', nan,
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán', 'Pennsylvania',
       'Texas', 'Utah', 'NV', 'Iowa', 'VA', 'TX', 'Indiana', 'Colorado',
       'Ohio', 'IN', 'New York', 'CA', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Virginia', 'Washington',
       'Missouri', 'California', 'NY', 'ND', 'MI', 'Florida', 'Michigan',
       'GA', 'New York, NY', 'Maine'], dtype=object)

6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [125]:
df['state_province'].isnull().sum()

2201

In [126]:
df["state_province"].replace(np.nan,'Unknown',inplace=True)

In [127]:
df['state_province'].isnull().sum()

0

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso, aquí la documentación. Para desarrollar este ejercicio deberéis:

- Sacar los valores únicos de la columna state_province.

In [128]:
df['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', 'Unknown',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán', 'Pennsylvania',
       'Texas', 'Utah', 'NV', 'Iowa', 'VA', 'TX', 'Indiana', 'Colorado',
       'Ohio', 'IN', 'New York', 'CA', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Virginia', 'Washington',
       'Missouri', 'California', 'NY', 'ND', 'MI', 'Florida', 'Michigan',
       'GA', 'New York, NY', 'Maine'], dtype=object)

- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:

In [129]:
cambios = {'NV':'Nevada','VA':'Virginia', 'TX':'Texas','IN':'Indianapolis','CA':'California','NY':'New York', 'ND':'North Dakota', 'MI':'Michigan',
       'GA':'Georgia', 'New York, NY':'New York','Ciudad Autónoma de Buenos Aires':'Buenos Aires'}

In [130]:
df['state_province'].replace(cambios,inplace=True)

In [131]:
df['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', 'Unknown',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Santiago Del Estero', 'Misiones', 'Catamarca',
       'Formosa', 'Jujuy', 'La Rioja', 'La Pampa', 'San Juan', 'San Luis',
       'Tucumán', 'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa',
       'Virginia', 'Indiana', 'Colorado', 'Ohio', 'Indianapolis',
       'New York', 'California', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Washington', 'Missouri',
       'North Dakota', 'Michigan', 'Florida', 'Georgia', 'Maine'],
      dtype=object)

In [132]:
# Esta limpiza se realiza para poder hacer la inserción de datos en MySQL ya que ' es un carácter especial.
df['name'] = df['name'].str.replace("'", '')

- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

In [133]:
provincias = list(df['state_province'].unique())

- Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [134]:
df_provincias = pd.DataFrame(provincias,columns=['state_province'])

In [135]:
def localizacion(elemento):
    """Esta función extrae la latitud y la longitud de una población dada.

        Args:
            elemento (str): población de la cual queremos saber sus coordenadas.

        Returns: devuelve la longitud y la latitud, o en el caso de que no haya población devuelve 0.0/0.0
        """
    if elemento != 'Unknown':
        geo = Nominatim(user_agent = 'Monica')
        localizacion = geo.geocode(elemento)
        return localizacion[1][0], localizacion[1][1]
    else:
        return 0.0, 0.0

In [136]:
df_provincias[['latitude','longitude']] = df_provincias['state_province'].apply(localizacion).apply(pd.Series)

Este ejercicio también se podría haber hecho con un diccionario.

In [137]:
df_provincias.head()

,state_province,latitude,longitude
0,Quebec,52.476089,-71.825867
1,Ontario,50.000678,-86.000977
2,Nova Scotia,45.196040,-63.165379
3,British Columbia,55.001251,-125.002441
4,Alberta,55.001251,-115.002136


- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [138]:
df_unido = df.merge(df_provincias,how='left',on = 'state_province')

In [139]:
df.shape

(2491, 5)

In [140]:
df_provincias.shape

(51, 3)

In [141]:
df_unido.shape

(2491, 7)

In [142]:
df_unido.isnull().sum()

web_pages         0
alpha_two_code    0
state_province    0
name              0
country           0
latitude          0
longitude         0
dtype: int64

In [143]:
df_unido.head()

,web_pages,alpha_two_code,state_province,name,country,latitude,longitude
0,https://www.cstj.qc.ca,CA,Quebec,Cégep de Saint-Jérôme,Canada,52.476089,-71.825867
1,https://www.lambtoncollege.ca,CA,Ontario,Lambton College,Canada,50.000678,-86.000977
2,http://www.acadiau.ca/,CA,Nova Scotia,Acadia University,Canada,45.196040,-63.165379
3,http://www.algonquincollege.com/,CA,Ontario,Algonquin College,Canada,50.000678,-86.000977
4,http://www.ashtoncollege.com/,CA,British Columbia,Ashton College,Canada,55.001251,-125.002441


8. Crea una BBDD en mysql que contenga las siguientes tablas:

In [144]:
def crear_bbdd(nombre_bbdd):
  ''' 
Esta función crea una base de datos con el nombre especificado.
Args:
    nombre_bbdd(str): nombre de la base de datos.
return: aunque no tiene return, crea una base de datos en MySQL.
  '''
  mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab")
  print("Conexión realizada con éxito")
    
  mycursor = mydb.cursor()

  try:
    mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
    print(mycursor)
  except mysql.connector.Error as err:
      print(err)
      print("Error Code:", err.errno)
      print("SQLSTATE", err.sqlstate)
      print("Message", err.msg)

In [145]:
crear_bbdd('evaluacion')

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS evaluacion..


In [146]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    ''' 
Esta función ejecuta una query dada en MySQL.
Args:
    nombre_bbdd(str): nombre de la base de datos.
    contraseña(str): nombre de la contraseña de MySQL.
    query(str): query que queremos ejecutar.
return: aunque no tiene return, ejecuta la query en MySQL, independiemente de que sea creación o insercción de tablas.
    '''
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
                                    

    mycursor = cnx.cursor()
    
  
    try: 
        mycursor.execute(query)
        cnx.commit() 
   
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

Aunque la creación de la base de datos y de las tablas se puede hacer utilizando el apartado models en MySQL, utilizamos funciones para demostrar las dos maneras (mirar archivo creacion_bbdd_tablas_modelo), aun así utilizaremos las querys proporcionadas para crearlas aquí.

In [147]:
query_paises = """
CREATE TABLE IF NOT EXISTS `paises` (
  `idestado` INT NOT NULL AUTO_INCREMENT,
  `nombre_pais` VARCHAR(45) NOT NULL,
  `nombre_provincia` VARCHAR(45) NOT NULL,
  `latitud` DECIMAL NOT NULL,
  `longitud` DECIMAL NOT NULL,
  PRIMARY KEY (`idestado`));"""

query_universidades = """
CREATE TABLE IF NOT EXISTS `universidades` (
  `iduniversidades` INT NOT NULL AUTO_INCREMENT,
  `nombre_universidad` VARCHAR(100) NOT NULL,
  `pagina_web` VARCHAR(100) NOT NULL,
  `paises_idestado` INT NOT NULL,
  PRIMARY KEY (`iduniversidades`),
  CONSTRAINT `fk_universidades_paises`
    FOREIGN KEY (`paises_idestado`)
    REFERENCES `paises` (`idestado`));"""

In [148]:
crear_insertar_tabla('evaluacion','AlumnaAdalab',query_paises)
crear_insertar_tabla('evaluacion','AlumnaAdalab',query_universidades)

9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs.

Hay una función que se ha utilizado durante el proceso de limpieza, que es:

In [149]:
def localizacion(elemento):
    """Esta función extrae la latitud y la longitud de una población dada.

        Args:
            elemento (str): población de la cual queremos saber sus coordenadas.

        Returns: devuelve la longitud y la latitud, o en el caso de que no haya población devuelve 0.0/0.0
        """
    if elemento != 'Unknown':
        geo = Nominatim(user_agent = 'Monica')
        localizacion = geo.geocode(elemento)
        return localizacion[1][0], localizacion[1][1]
    else:
        return 0.0, 0.0

El resto de proceso de limpieza se ha juntado en una sola función, que necesita de la anterior para así unificar todo el proceso en una sola función:

In [150]:
def limpieza(dataframe):
    """Esta función incluye todo el proceso de limpiza del dataframe.
    Args:
        dataframe(dataframe): el dataframe el cual queremos limpiar.
    return: esta función devuelve el dataframe limpio.
    """
    nuevas_columna = {col: col.replace("-", "_") for col in dataframe.columns}
    dataframe.rename(columns= nuevas_columna, inplace = True)

    dataframe.drop('domains',axis=1,inplace=True)

    df_explode = dataframe.explode('web_pages')
    dataframe = df_explode

    duplicados = dataframe.duplicated(subset='name').sum()
    if duplicados != 0:
        dataframe.drop_duplicates(subset='name',inplace=True)

    dataframe['state_province'].replace({None},np.nan,inplace=True)
    dataframe['state_province'].replace(np.nan,'Unknown',inplace=True)

    cambios = {'NV':'Nevada','VA':'Virginia', 'TX':'Texas','IN':'Indianapolis','CA':'California','NY':'New York', 'ND':'North Dakota', 'MI':'Michigan',
    'GA':'Georgia', 'New York, NY':'New York','Ciudad Autónoma de Buenos Aires':'Buenos Aires'}
    dataframe['state_province'].replace(cambios,inplace=True)

    dataframe['name'] = dataframe['name'].str.replace("'", '')

    provincias = list(dataframe['state_province'].unique())
    df_provincias = pd.DataFrame(provincias,columns=['state_province'])
    df_provincias[['latitude','longitude']] = df_provincias['state_province'].apply(localizacion).apply(pd.Series)
    df_unido = dataframe.merge(df_provincias,how='left',on = 'state_province')
    return df_unido

El apartado 9 y 10 (bonus) se han hecho solo a partir de funciones ya definidas en sus propios apartados.

10. BONUS
- Introduce los datos en la BBDD de SQL.

In [151]:
def check_provincias(nombre_bbdd):
    """ Esta función extrae todas las provincias que hay una tabla de MySQL.
        Args:
            nombre_bbdd(str): nombre de la base de datos donde se encuentra la tabla.
        return: devuelve una lista de tuplas con todos los nombres de las provincias que hay en la tabla.
        """
    cnx = mysql.connector.connect(user='root', password=f'AlumnaAdalab',
                                  host='127.0.0.1', database=f"{nombre_bbdd}",  auth_plugin = 'mysql_native_password')
    mycursor = cnx.cursor()


    query_existe_provincia = f"""
            SELECT DISTINCT nombre_provincia FROM paises
            """
    mycursor.execute(query_existe_provincia)
    provincia = mycursor.fetchall()
    return provincia

In [152]:
# Podriamos poner un else print(x), pero el output sería muy grande y no lo ponemos.
for indice, fila in df_unido.iterrows():
   
    query_paises = f"""
                INSERT INTO paises (nombre_pais, nombre_provincia, latitud, longitud) 
                VALUES ("{fila['country']}", "{fila['state_province']}","{fila['latitude']}","{fila['longitude']}");
                """
    
    provincias = check_provincias('evaluacion')
    lista_provincias = [i[0] for i in provincias]
    if len(provincias) == 0 or fila['state_province'] not in lista_provincias: 
        crear_insertar_tabla("evaluacion","AlumnaAdalab", query_paises)

In [153]:
def sacar_id_estado(estado):
    """Esta función extrae el id de un estado dado.
    Args:
        estado (str): el estado del cual queremos saber su id.

    Returns: devuelve el id del estado especificado.
    """
    mydb = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                        host='127.0.0.1', database='evaluacion')
    mycursor = mydb.cursor()

    try:
        query_sacar_id = f"SELECT idestado FROM paises WHERE nombre_provincia = '{estado}'"
        mycursor.execute(query_sacar_id)
        id_ = mycursor.fetchall()[0][0]
        return id_
    
    except: 
            return "Sorry, no tenemos ese estado en la BBDD y por lo tanto no te podemos dar su id."

In [154]:
def check_universidades(nombre_bbdd):
    """ Esta función extrae todas las universidades que hay una tabla de MySQL.
        Args:
            nombre_bbdd(str): nombre de la base de datos donde se encuentra la tabla.
        return: devuelve una lista de tuplas con todos los nombres de las universidades que hay en la tabla.
        """
    cnx = mysql.connector.connect(user='root', password=f'AlumnaAdalab',
                                  host='127.0.0.1', database=f"{nombre_bbdd}",  auth_plugin = 'mysql_native_password')
    mycursor = cnx.cursor()


    query_existe_uni = f"""
            SELECT DISTINCT nombre_universidad FROM universidades
            """
    mycursor.execute(query_existe_uni)
    unis = mycursor.fetchall()
    return unis

In [155]:
for indice, fila in df_unido.iterrows():

    query_uni = f"""
                INSERT INTO universidades (nombre_universidad, pagina_web, paises_idestado) 
                VALUES ('{fila["name"]}', '{fila["web_pages"]}', '{sacar_id_estado(fila["state_province"])}');
                """
    universidades = check_universidades('evaluacion')
    lista_universidades = [i[0] for i in universidades]
    if len(universidades) == 0 or fila['name'] not in lista_universidades: 
        crear_insertar_tabla('evaluacion','AlumnaAdalab',query_uni)